In [2]:
import re
import nltk
import string
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

# I. Data preparation

To learn to tokenize words, we first have a text to learn the most common words. Here I use the wikitext103 dataset from https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/.

In [3]:
wikitext = open('C:\\Users\\TA\\Downloads\\wiki.train.tokens', encoding = 'utf8').read()

In [4]:
len(wikitext)

10780437

In [5]:
wikitext[:50]

' \n = Valkyria Chronicles III = \n \n Senjō no Valkyr'

Then we do a simple word tokenization and compute the probability of each words.

In [6]:
wikitext = re.compile(r'[.?!]').sub('_S_', wikitext)
words = re.findall(re.compile(r'\w+'), wikitext)
n = len(words)
n

1827460

In [7]:
bow = Counter(words)
bow.most_common(10)

[('the', 113161),
 ('_S_', 77305),
 ('of', 56889),
 ('unk', 54625),
 ('and', 50603),
 ('in', 39486),
 ('to', 39190),
 ('a', 34250),
 ('was', 20985),
 ('The', 17602)]

In [8]:
unigram = dict(zip(words, [bow[word]/n for word in words]))
unigram['the']

0.0619225591805019

# II. Finding a suitable segmenter

## 1. Greedy Algorithm

Now come to the main part, tokenize a continous alphabetical character string into words with the vocabulary we have. First I try a longest-match-first greedy algorithm called maxmatch.

In [9]:
def maxmatch(text, desc=False):
    '''If you want to take the longest word from left to right, set desc to true. 
    If you take the longest word from right to left, set desc to false.'''
    if not text:
        return []
    if (desc):
        for i in range(len(text),-1,-1):
            firstword = text[:i]
            remainder = text[i:len(text)]
            if firstword in words:
                return [firstword] + maxmatch(remainder, desc=True)
    else:
        for i in range(len(text)+1):
            firstword = text[:i]
            remainder = text[i:len(text)]
            if remainder in words:
                return maxmatch(firstword, desc=False) + [remainder]
    return [text]

Let's try it with some phrase!

In [10]:
maxmatch("gotoschool", desc=False)

['go', 'to', 'school']

In [11]:
maxmatch("Iamastudent", desc=True)

['I', 'am', 'as', 't', 'u', 'dent']

In [12]:
maxmatch("Iamastudent", desc=False)

['I', 'am', 'a', 'student']

In [13]:
maxmatch("speedofart", desc=False)

['speed', 'of', 'art']

In [14]:
maxmatch("smallandinsignificant", desc=False)

['s', 'mal', 'land', 'insignificant']

## 2. N-gram Language Model 

Another way to tokenize words is to accept the bag-of-word assumption and choose the phrase with highest probability. Now we will try it out.

In [15]:
def unigramProb(words):
    '''Compute the probility of a word sequence.'''
    prob = 1
    for word in words:
        if word in unigram:
            prob *= unigram[word]
        else:
            prob *= 0
            break
    return prob

In [16]:
def memo(func):
    '''A decorator that kinda saves the results to a cache'''
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [17]:
@memo
def ngramModel(text, probFunction):
    '''A n-gram model for word tokenization'''
    if not text:
        return []
    candidate = [[text[:i+1]] + ngramModel(text[i+1:], probFunction) for i in range(len(text)+1)]
    return max(candidate, key=probFunction)

In [18]:
ngramModel('iamastudent', unigramProb)

['i', 'am', 'a', 'student']

In [19]:
ngramModel('smallandinsignificant', unigramProb)

['small', 'and', 'insignificant']

In [20]:
ngramModel('Turnoffthelightbeforegoingout', unigramProb)

['T', 'urn', 'off', 'the', 'light', 'before', 'going', 'out']

The result is pretty good but the model still depends the bag-of-word assumption and new words. How about using bigram with smoothing (we can't use simple smoothing technique like add-k because it can't distinguish likely unknown from unlikely unknown words)

In [21]:
twowords = list(nltk.bigrams(words))
m = len(twowords)
m

1827459

In [22]:
bigrams = Counter(twowords)
bigrams.most_common(10)

[(('of', 'the'), 16991),
 (('_S_', 'The'), 13635),
 (('in', 'the'), 10580),
 (('to', 'the'), 5911),
 (('unk', 'unk'), 5722),
 (('_S_', 'In'), 4930),
 (('unk', '_S_'), 4607),
 (('unk', 'and'), 4272),
 (('and', 'the'), 4259),
 (('on', 'the'), 4194)]

In [24]:
bigram = dict(zip(twowords, [bigrams[twoword]/m for twoword in twowords]))
(bigrams['I', 'am'], bigram['I', 'am'])

(55, 3.0096434448050544e-05)

In [28]:
def bigramProbLTR(words):
    '''Compute the probility of a word sequence with bigram and stupid back-off from left to right.'''
    prob = 1
    i = 0
    for word in words:
            if i == 0: 
                prev = '_S_'
            else:
                prev = words[i-1]
            if (prev + ' ' + word) in bigram and prev in unigram:
                prob *= bigram[prev + ' ' + word]
            else:
                if word in unigram:
                    prob *= unigram[word]*0.4
                else:
                    prob *= 0
                    break
            i+=1
    return prob

In [29]:
def bigramProbRTL(words):
    '''Compute the probility of a word sequence with bigram and stupid back-off from right to left.'''
    prob = 1
    i = -1
    for word in words:
            if i == -1: 
                following = '_S_'
            else:
                following = words[i+1]
            if (word + ' ' + following) in bigram and prev in unigram:
                prob *= bigram[word + ' ' + following]
            else:
                if word in unigram:
                    prob *= unigram[word]*0.4
                else:
                    prob *= 0
                    break
            i-=1
    return prob

In [30]:
ngramModel('smallandinsignificant', bigramProbLTR)

['small', 'and', 'insignificant']

In [31]:
ngramModel('Wouldyoulikeacupoftea',bigramProbRTL)

['Would', 'you', 'like', 'a', 'cup', 'of', 'tea']

In [32]:
ngramModel('adrybaresandyholewithnothinginittositdownonortoeat', bigramProbRTL)

['a',
 'dry',
 'bare',
 'sandy',
 'hole',
 'with',
 'nothing',
 'in',
 'it',
 'to',
 'sit',
 'down',
 'on',
 'or',
 'to',
 'eat']

## 3. Smoothing with NLTK 

Bigram model seems better than unigram, but still not really good because we still can't handle oov words. We can do some smoothing (Katz backoff, Kneser-Ney,...) to have better results. For example, now I will use nltk to make a bigram model without and with Kneser-Ney smoothing and see the result.

In [87]:
from nltk.lm import NgramCounter, Vocabulary, MLE, KneserNeyInterpolated
newVocab = Vocabulary(bow)
count = NgramCounter([list((word,) for word in words) + list(nltk.bigrams(words))])
simpleModel = MLE(2, vocabulary=newVocab, counter=count)
print(simpleModel.counts)

<NgramCounter with 2 ngram orders and 3654919 ngrams>


In [88]:
def newProb(words):
    '''Another way to compute the log probability of a word sequence with bigram.'''
    logprob = 0
    bigramText = [('_S_', words[0])] + list(nltk.bigrams(words))
    for ngram in bigramText:
        logprob += simpleModel.logscore(ngram[0], [ngram[1]])
    return logprob

In [89]:
ngramModel('gotoschool', newProb)

['go', 'to', 'school']

In [90]:
ngramModel('speedofart', newProb)

['s', 'p', 'e', 'e', 'd', 'of', 'art']

In [91]:
smoothedModel = KneserNeyInterpolated(2, vocabulary=newVocab, counter=count)
len(smoothedModel.counts)

2

In [92]:
def smoothedProb(words):
    '''Compute the log probability of a word sequence with Kneser-Ney Interpolated.'''
    logprob = 0
    bigramText = [('_S_', words[0])] + list(nltk.bigrams(words))
    for ngram in bigramText:
        logprob += smoothedModel.logscore(ngram[0], [ngram[1]])
    return logprob

In [93]:
ngramModel('gotoschool', smoothedProb)

['gotoschool']

In [94]:
smoothedProb(['gotoschool'])

-15.017895817356235

In [95]:
smoothedProb(['go', 'to', 'school'])

-20.304866550884615

Using nltk to make a n-gram model with Kneser-Ney smoothing leads to a very bad result. Maybe there's something wrong with my code, but for now we only have good enough results from unigram and bigram model.  

# III. Evaluation

To evaluate precisely each algorithm, we can use a test set. But how to use it efficiently will take a bit of time.

In [34]:
testset = open('C:\\Users\\TA\\Downloads\\wiki.test.tokens', encoding = 'utf8').read()
len(testset)

1255018

In [65]:
tests = re.split(r'[.?!\n]', testset)
tests = [sentences for sentences in tests if len(sentences) > 30]
len(tests)

9668

The testset has about 10000 sentences, that's a lot. We can test by running the model with the first 1000 sentences and getting the number of sentences that they guess right.

In [98]:
def test_model(tests, otherParam):
    "Try segmenter on tests; report failures; return fraction correct."
    return sum([test_one_sentence(test, otherParam) 
               for test in tests]), len(tests)

def test_one_sentence(test, otherParam):
    words = re.findall(re.compile(r'\w+'), test)
    result = ngramModel(''.join(words), otherParam)
    correct = (result == words)
    return correct

In [99]:
test_model(tests[:1000], unigramProb)

(862, 1000)

In [100]:
test_model(tests[:1000], bigramProbRTL)

(861, 1000)

In [101]:
test_model(tests[:1000], bigramProbLTR)

(861, 1000)

Surprisingly, the unigram model have a better result than the bigram model. Look like a simple model with enough data is good enough for segmentation task. Of course this still have quite a lot of wrong assumption, but the unigram model is still the best model we have for now. 